In [ ]:
# imports
import networkx as nx
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split
from pgmpy.models import BayesianModel
from pgmpy.independencies import Independencies
from scipy.stats import chi2_contingency

import dowhy.gcm as gcm

In [ ]:
# DAG describing this data generating process:
G = nx.DiGraph()
# G.add_edges_from([
#     ('A', 'G'),
#     ('B', 'G'),
#     ('G', 'H'),
#     ('C', 'F'),
#     ('D', 'F'),
#     ('E', 'F'),
#     ('F', 'H'),
#     ('B', 'H'),
# ])

G.add_edges_from([
    ('X', 'A'),
    ('g', 'A'),
    ('g', 'B'),
    ('Y', 'B'),
])

In [ ]:
# ## Data Generation
# def data_gen_process(seed=42, n_samples=1000):
#     np.random.seed(seed)

#     A = np.random.binomial(1, 0.5, n_samples)
#     B = np.random.normal(0, 1, n_samples)
#     C = np.random.normal(0, 1, n_samples)
#     D = np.random.normal(0, 1, n_samples)
#     E = np.random.normal(0, 1, n_samples)

#     G = np.tanh(A + 0.5 * B) + np.random.normal(0, 0.5, n_samples)  # Increase noise
#     F = np.sin(C) + np.log(np.abs(D) + 1) + E**2 + np.random.normal(0, 0.5, n_samples)  # Increase noise

#     H_base = G**2 + 2 * np.cos(F) + 0.5 * B**2 + np.random.normal(0, 0.5, n_samples)  # More complex non-linear relationship
#     H = np.where(B > 0, H_base - 1.5 * G, H_base + 1.5 * G)  # Simpson's effect

#     Z = np.random.binomial(1, 0.5, n_samples)
#     H += Z * np.random.normal(0, 0.5, n_samples)

#     # Discretize H
#     discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
#     H_discrete = discretizer.fit_transform(H.reshape(-1, 1)).flatten()

#     return pd.DataFrame({
#         'A': A,
#         'B': B,
#         'C': C,
#         'D': D,
#         'E': E,
#         'G': G,
#         'F': F,
#         'H': H_discrete,
#         'Z': Z 
#     })

# df = data_gen_process()
# # Preprocess the data to include only the relevant variables based on the DAG
# relevant_variables = ['A', 'B', 'C', 'D', 'E', 'G', 'F', 'Z']
# X = df[relevant_variables]
# y = df['H']

In [ ]:
# Data Generation
def data_gen_process(seed=42, n_samples=1000):
    np.random.seed(seed)

    g = np.random.normal(1, 0.5, n_samples)
    X = np.random.normal(0, 1, n_samples)
    Y = np.random.normal(0, 1, n_samples)
    A = np.tanh(X + 0.5 * g) + np.random.normal(0, 0.5, n_samples)
    B = np.sin(g) + np.log(np.abs(g) + 1) + Y**2 + np.random.normal(0, 0.5, n_samples)
    
    return pd.DataFrame({
        'g': g,
        'X': X,
        'Y': Y,
        'A': A,
        'B': B,
    })

df = data_gen_process()
# Preprocess the data to include only the relevant variables based on the DAG
relevant_variables = ['X', 'Y', 'A'] # 'g' hidden
X = df[relevant_variables]
y = df['B']

## Cond Indep Test
(using the kernel dependence measure?)

In [33]:
model = BayesianModel(list(G.edges()))
independencies = model.get_independencies() # can reduce a lot of redundancies
for indep in independencies.get_assertions():
    print(indep, ":", indep.event1, indep.event2, indep.event3)
    # print(gcm.independence_test(df[indep.event1].to_numpy(), 
    #                       df[indep.event2].to_numpy(), 
    #                       conditioned_on=df[indep.event3].to_numpy()))
    break

(A ⟂ Y) : frozenset({'A'}) frozenset({'Y'}) frozenset()


In [94]:
# gcm.independence_test(df.A.to_numpy(), df.Y.to_numpy())
# gcm.independence_test(np.random.normal(1, 2, 1000000), np.random.normal(0, 1, 1000000))
gcm.independence_test(np.random.poisson(1, 1000000), np.random.normal(1, 2, 1000000))

In [ ]:
model = BayesianModel(list(G.edges()))
independencies = model.get_independencies() # can reduce a lot of redundancies

indep_constraints_status = []
for indep in independencies.get_assertions():
    val = gcm.independence_test(df[indep.event1].to_numpy(), 
                          df[indep.event2].to_numpy(), 
                          conditioned_on=df[indep.event3].to_numpy())
    print(indep, val)
    indep_constraints_status.append([indep,val])

indep_constraints_status

## Penalty Calculation (Loss func)

In [ ]:
def calculate_penalty(predictions, variables, states):
    penalty = 0
    
    # Example calculation for sum-to-one constraint
    for node in variables:
        sum_probs = tf.reduce_sum([predictions[node][state] for state in states[node]], axis=0)
        penalty += tf.reduce_mean(tf.square(sum_probs - 1))  # Sum-to-one constraint penalty
    
    # Example calculation for non-negativity constraint
    for node in variables:
        for state in states[node]:
            penalty += tf.reduce_mean(tf.nn.relu(-predictions[node][state]))  # Non-negativity constraint penalty
    
    return penalty

In [ ]:
def calculate_penalty(predictions, X, model):   
    constraints_penalty = 0

    # Conditional Independences
    independencies = model.get_independencies()
    for independence in independencies.get_assertions():
        var1 = independence.event1
        var2 = independence.event2
        ...

    # Sum-to-One Constraints
    if np.any(np.abs(np.sum(predictions, axis=1) - 1) > 1e-5):
        constraints_penalty += np.sum(np.abs(np.sum(predictions, axis=1) - 1))

    # Non-negativity Constraints
    if np.any(predictions < 0):
        constraints_penalty += np.sum(predictions[predictions < 0]**2)

    return constraints_penalty

# Custom loss function to include the DAG-induced constraints
def custom_loss(y_true, y_pred):
    base_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    penalty = calculate_penalty(y_pred, X, BayesianModel(list(G.edges())))    
    return base_loss + penalty

# Define the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, input_dim=X.shape[1], activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss=custom_loss, metrics=['accuracy'])

# Training the model
model.fit(X, y, epochs=10, batch_size=32)

In [ ]:
def calculate_penalty(predictions, variables, states):   
    model = BayesianModel(list(G.edges()))
    constraints = []

    # Conditional Independences
    independencies = model.get_independencies()
    # for independence in independencies.get_assertions():
    #     constraints.append(f"{independence}")
    



    # Sum-to-One Constraints
    # ...
       
    # Non-negativity Constraints
    # ...


    # return constraints
    for constraint in constraints:
        print(constraint)

    penalty = ...

    return penalty

In [ ]:
# custom loss function to include the DAG-induced constraints
def custom_loss(y_true, y_pred):
    base_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    penalty = calculate_penalty(predictions, variables, states)
    return base_loss + penalty

# Define the neural network model
model = Sequential([
    Dense(64, input_dim=X.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss=custom_loss, metrics=['accuracy'])

In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Train the model #bayesian hyperparameter optimization?
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)
print("Done fitting.")

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

## to identify DAG induced constraints:

In [ ]:
def generate_constraints(model):


    constraints = []
    
    # Conditional Independences
    independencies = model.get_independencies()
    for independence in independencies.get_assertions():
        constraints.append(f"{independence}")
    
    # # Sum-to-One Constraints
    # for node in model.nodes():
    #     states = ['x1', 'x2', 'x3']  # Example states, should be defined based on your data
    #     sum_to_one = ' + '.join([f'P({node}={state})' for state in states]) + ' = 1'
    #     constraints.append(sum_to_one)
    
    # # Non-negativity Constraints
    # for node in model.nodes():
    #     states = ['x1', 'x2', 'x3']  # Example states, should be defined based on your data
    #     for state in states:
    #         non_negativity = f'P({node}={state}) >= 0'
    #         constraints.append(non_negativity)
    
    return constraints

# Example usage
constraints = generate_constraints(model)
for constraint in constraints:
    print(constraint)


In [ ]:
model = BayesianModel(list(G.edges()))


# conditional independences
independencies = model.get_independencies()
print(independencies)

In [ ]:
# Example: If you have a distribution P(X)
# The sum-to-one constraint for a discrete variable X with states {x1, x2, x3} is:
sum_to_one_constraint = 'P(x1) + P(x2) + P(x3) = 1'
print(sum_to_one_constraint)


# Example: For a probability P(X=x)
non_negativity_constraints = ['P(x) >= 0']
print(non_negativity_constraints)